# Modeling the Müller-Brown Potential Using a Neural Network

In this tutorial, we will learn how to use a neural network (NN) model to predict the energy of points on the Müller-Brown potential energy surface.


In [ ]:
from math import exp, pow
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

## Defining the Müller-Brown Potential

The Müller-Brown potential is a well-known model potential energy surface used in computational chemistry to study reaction dynamics and transition states. It is defined by a sum of four Gaussian-like terms, and its mathematical form is (for more details see [here](https://www.wolframcloud.com/objects/demonstrations/TrajectoriesOnTheMullerBrownPotentialEnergySurface-source.nb)): 

$ V(x,y) = \sum_{k=1}^4 A_k \mathrm{exp}\left( a_k \left( x - x_k^0 \right)^2 + b_k \left( x - x_k^0 \right) \left( y - y_k^0 \right) + c_k \left( y - y_k^0 \right)^2 \right) $.

First, we define the Müller-Brown potential as a Python function:

In [ ]:
def mueller_brown_potential(x, y):
    """
    Compute the Müller-Brown potential energy function at position (x, y).

    Parameters:
    -----------
    x : float
        x-coordinate of the point.
    y : float
        y-coordinate of the point.

    Returns:
    --------
    value : float
        Potential energy at the given (x, y) position.
    """
    # Amplitude parameters
    A = [-200, -100, -170, 15]
    
    # Shape coefficients
    a = [-1, -1, -6.5, 0.7]
    b = [0, 0, 11, 0.6]
    c = [-10, -10, -6.5, 0.7]
    
    # Center coordinates of the minima
    x_0 = [1.0, 0.0, -0.5, -1.0]
    y_0 = [0.0, 0.5, 1.5, 1.0]

    v_xy = 0.0

    for k in range(4):
        dx = x - x_0[k]
        dy = y - y_0[k]
        # Scale the function by 0.1 to make visualization easier
        v_xy += 0.1 * A[k] * np.exp(a[k] * dx**2 + b[k] * dx * dy + c[k] * dy**2)

    return v_xy

## Generate Training Data

Next, we need to generate data points to train the neural network. The training data will be generated using the Müller-Brown Potential and a range of x and y values.

In [ ]:
# Generate a set of x and y values
x = np.arange(-1.8, 1.4, 0.05)
y = np.arange(-0.8, 2.4, 0.05)

# Create a rectangular grid out of two given one-dimensional xs and ys arrays
X, Y = np.meshgrid(x, y)

xy, v = [], []
xy_truncated, v_truncated = [], []

for j in y:
  for i in x:
    xy.append([i,j])
    e = mueller_brown_potential(i,j)
    v.append(e) # Storing potential energy values in the v array
    if e < 20:  # Keeping only low-energy points for training
      xy_truncated.append([i,j])
      v_truncated.append(e)

# Reshape e array so that we can plot our data on a 2D surface that is len(xs) by len(ys)
V = np.reshape(v,(len(y),-1))

# Print some statistics
print("V_min:", np.amin(V), "V_max:", np.amax(V))
print("Size of dataset:", len(v))
print("Size of truncated dataset:", len(v_truncated))

### Visualizing Data: 3D Surface

We will now create a 3D plot of our data. To make the plot more readable, we will exclude the points that have extremely high energy.

In [ ]:
fig = go.Figure(data=[go.Surface(z=V, x=X, y=Y, colorscale='rainbow', cmin=-15, cmax=9)])

# Add contour lines
fig.update_traces(contours_z=dict(show=True, project_z=True, width=1))

# Format the layout
fig.update_layout(
    title='Müller-Brown PES', 
    width=600, height=600,
    scene = dict(
        zaxis_title="V(x,y)",
        zaxis = dict(dtick=3, range=[-15, 15]),
        camera_eye = dict(x=-1.2, y=-1.2, z=1.0)
    )
)

fig.show()

### Visualizing Data: 2D Contour Surface

To allow for an easier visualization of the potential energy surface, we can generate a 2D contour surface.

In [ ]:
# Create the figure
fig = plt.figure(figsize=(6, 5))

# Define contour levels
levels = np.arange(-12, 8, 2)

# Contour lines
contour_lines = plt.contour(X, Y, V, levels=levels, colors='k', linewidths=1.0)
plt.clabel(contour_lines, inline=True, fmt='%3.0f', fontsize=8)

# Filled contours
contour_fill = plt.contourf(X, Y, V, levels=levels, cmap='rainbow', extend='both', vmin=-15, vmax=8)

# Axis labels and ticks
plt.xlabel("x", labelpad=2.5)
plt.ylabel("y", labelpad=2.5)
plt.tick_params(axis='both', pad=2, labelsize=8)

# Colorbar
cbar = plt.colorbar(contour_fill)
cbar.ax.tick_params(labelsize=8)

# Title
plt.title('Müller-Brown Contour Surface', fontsize=10)

# Layout
plt.tight_layout()
plt.show()

## Loading PyTorch and Training Data

After installing and importing pytorch, we will convert our truncated dataset to a tensor data object. Then, randomly select 80% of the data points for training. 


In [ ]:
import torch
import torch.nn as nn
from torch import Tensor
from torch.utils.data import TensorDataset, DataLoader, Subset

# convert the dataset to TensorDataset
dataset = TensorDataset(Tensor(xy_truncated), Tensor(v_truncated))

# Randomly select 80% of the data points
train_indices = torch.randint(0, len(dataset), (int(len(dataset)*0.8),))
train_dataset = Subset(dataset, train_indices)

# Prepare a DataLoader object using the selected training set
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
print("Size of training set:", len(train_loader.dataset))

### Defining the Neural Network Class

Here we define our neural network as a Python class. A function ($\it{train\_loop}$) is used to loop through our training data.

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, n=20):  # n is the number of neurons of the hidden layer(s)
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2,n), # Linear function taking 2 inputs and outputs data for n neurons
            nn.Tanh(),      # Non-linear activation function
            nn.Linear(n, 1) # Linear function taking data from n neurons and producing one output
        )

    def forward(self, x):
        return self.model(x)

def train_loop(dataloader, model, loss_fn, optimizer, epoch):

    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute model's prediction and loss
        pred = model(X)
        loss = loss_fn(pred.squeeze(), y)

        # Backpropagation - using the gradients of the loss function to update the weights and biases
        optimizer.zero_grad() # Zero out the gradients from the previous iteration
        loss.backward()       # Compute the gradients of the loss function
        optimizer.step()      # Update the weights and biases using the gradients

        if batch % 32 == 0 and epoch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"epoch: {epoch:>4d} loss: {loss:>7.3f}  [{current:>5d}/{size:>5d}]")

## Training the Model

Now we can train the neural network. We will finish our training when the desired number of epochs has been reached. We will also define other hyper-parameters used for the training.

In [ ]:
n_hidden = 20
learning_rate = 1e-2
epochs = 800

model = NeuralNetwork(n_hidden)
loss_fn = nn.functional.mse_loss
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    train_loop(train_loader, model, loss_fn, optimizer, epoch)

print("Done with Training!")

## Plotting Reference, Predicted, and Difference Surfaces

Finally, we will plot the Müller-Brown potential energy surface using the analytical function (reference), using the predicted values from the neural network (predicted), and we will show the difference between the predicted and reference surfaces.

In [ ]:
def show_surface(model):

  v_pred = model(Tensor(xy))
  V_pred = np.reshape(v_pred.detach().numpy(),(len(y),-1))
  Vdiff = np.subtract(V_pred, V)

  fig = plt.figure(figsize=(6, 15))
  
  levels = np.arange(-12, 8, 2)

  plt.subplot(3, 1, 1)
  ct = plt.contour(X, Y, V, levels, colors='k', linewidths=1.0)
  plt.clabel(ct, inline=True, fmt='%3.0f', fontsize=8)
  ct = plt.contourf(X, Y, V, levels, cmap=plt.cm.rainbow, extend='both', vmin=-12, vmax=8)
  plt.title("(a) Reference", fontsize=12)
  plt.xlabel("x", labelpad=2.5)
  plt.ylabel("y", labelpad=2.5)
  plt.tick_params(axis='both', pad=2, labelsize=8)
  cbar= plt.colorbar()
  cbar.ax.tick_params(labelsize=8)

  plt.subplot(3, 1, 2)
  ct = plt.contour(X, Y, V_pred, levels, colors='k', linewidths=1.0)
  plt.clabel(ct, inline=True, fmt='%3.0f', fontsize=8)
  ct = plt.contourf(X, Y, V_pred, levels, cmap=plt.cm.rainbow, extend='both', vmin=-12, vmax=8)
  plt.title("(b) Predicted", fontsize=12)
  plt.xlabel("x", labelpad=2.5)
  plt.ylabel("y", labelpad=2.5)
  plt.tick_params(axis='both', pad=2, labelsize=8)
  cbar= plt.colorbar()
  cbar.ax.tick_params(labelsize=8)

  levels = np.arange(-4, 4, 2)
  
  plt.subplot(3, 1, 3)
  ct = plt.contour(X, Y, Vdiff, levels, colors='k', linewidths=1.0)
  plt.clabel(ct, inline=True, fmt='%3.0f', fontsize=8)
  ct = plt.contourf(X, Y, Vdiff, levels, cmap=plt.cm.rainbow, extend='both', vmin=-4, vmax=4)
  plt.title("(c) Difference", fontsize=12)
  plt.xlabel("x", labelpad=2.5)
  plt.ylabel("y", labelpad=2.5)
  plt.tick_params(axis='both', pad=2, labelsize=8)
  cbar= plt.colorbar()
  cbar.ax.tick_params(labelsize=8)

  plt.tight_layout()

  plt.show()

show_surface(model)